In [1]:

from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import os
from dotenv import load_dotenv, find_dotenv
import time
DEBUG=0

In [2]:
def get_bing_key():
    load_dotenv(find_dotenv())
    # load environment variables
    BING_KEY = os.getenv("BING_KEY")
    DEBUG = os.getenv("DEBUG")
    return BING_KEY

In [3]:
def get_lats_longs(address):
    '''This function uses GeoCoder to retrieve the latitude and longitude for neighbourhoods.
    address to look up
    returns lat and long
    '''
    BING_KEY = get_bing_key()
    # # get geocode for postal codes
    # initialize your variable to None
    lat_lng_coords = None
    lat=None
    long=None
    max_loops = 10
    curr_loop =0
    # loop until you get the coordinates (sometime dont' get a response)
    lat_lng_coords = None
    while(lat_lng_coords is None and curr_loop < max_loops):
        g = geocoder.bing('{}'.format(address),key=BING_KEY)
        if DEBUG > 0:
            print (g)
        lat_lng_coords = g.latlng
        if DEBUG > 0:
            print(lat_lng_coords)
        if lat_lng_coords == None:
            time.sleep(1)
        curr_loop += 1

    lat=lat_lng_coords[0]
    long=lat_lng_coords[1]

    return lat,long

In [8]:
def update_missing_coords():
    '''
    Find all rows in PointsofINterest table that are missing lat and long
    Lookup address using Geocoder and update database row
    '''
    null_lats = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None)
    for row in null_lats:
        #null_lat = session.query(PointsOfInterest).get(1)
        lat, long= get_lats_longs(row.address)
        props = {'latitude':lat, 'longitude':long}

        for key, value in props.items():
            setattr(row, key, value)

    # save up our commits
    session.commit()
    session.flush()
    return True


In [9]:
db=connect_db() #establish connection / creates database on first run
Session = sessionmaker(bind=db)
session = Session()
update_missing_coords()

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=22-24+Tiverton+Avenue+Riverdale+Toronto&o=json&inclnb=1&key=AjSRA1cu_X4uwtkQI4EfWAckPz7ipmlB-HHNzf5Zx0ZW8FiLwMwo8BZXYOpkbQrU&maxResults=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002275F0FAE10>, 'Connection to dev.virtualearth.net timed out. (connect timeout=5.0)'))


True

In [11]:
session.query(PointsOfInterest).get(3500).latitude #43.66449

43.64548

In [13]:
session.commit()
session.flush()
session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).count()

0